In [ ]:
from src.experiments.common import Experiment, Run, get_run_from_path
from pathlib import Path
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd
import seaborn as sns
experiment = Experiment("sghmc_gradients")

In [ ]:
run = get_run_from_path(Path("../experiment_results/sghmc_gradients/2021-10-18/10-55-19").resolve())

In [ ]:
variance_estimates = run.runs[5].load_result("variance_estimates")
global_steps = run.runs[5].load_result("global_steps")

In [ ]:
import torch
def subsample(tensor, n, dim=0, seed=123):
    with torch.random.fork_rng():
        torch.manual_seed(seed)
        index = torch.randperm(tensor.shape[dim])[:n]
    return tensor.index_select(dim, index), index

In [ ]:
n_steps = 100
start = (len(global_steps) - n_steps) // 2
stop = len(global_steps) - start
slice_ = slice(start, stop)
step = global_steps[slice_]

In [ ]:
variance_inter_batch = variance_estimates["variance_inter_batch"][slice_]
variance_estimated = variance_estimates["variance_estimated"][slice_]

variance_inter_batch, sample_idx = subsample(variance_inter_batch, 500, 1)
variance_estimated = variance_estimated[:, sample_idx]


In [ ]:
is_zero = variance_estimates["variance_estimated"].isclose(torch.tensor(0.0)).all(0)
is_zero |= variance_estimates["variance_inter_batch"].isclose(torch.tensor(0.0)).all(0)

non_zero_index = (~is_zero).nonzero().flatten()
plot_idx = non_zero_index[torch.randperm(len(non_zero_index))[:9]]

plot_data = pd.concat(
    [
        pd.DataFrame(
            {
                "Interbatch variance": variance_estimates["variance_inter_batch"][
                    :, idx
                ],
                "Estimated variance": variance_estimates["variance_estimated"][:, idx],
            },
            index=pd.MultiIndex.from_product(
                [[idx.item()], global_steps], names=("index", "step")
            ),
        )
        for idx in plot_idx
    ]
).assign(step_mod_100=lambda x: x.index.get_level_values("step") % 100)


In [ ]:
pd.Series(plot_idx).map(get_data)

In [ ]:
def get_data(idx):

    return pd.DataFrame(
        {
            "Inter batch variance": variance_estimates["variance_inter_batch"][:, idx],
            "Estimated variance": variance_estimates["variance_estimated"][:, idx],
        },
        index=pd.MultiIndex.from_product(
            [[idx.item()], global_steps], names=("index", "step")
        ),
    )


fg = (
    pd.concat(map(get_data, plot_idx))
    .assign(step_mod_100=lambda x: x.index.get_level_values("step") % 100)
    .reset_index()
    .sample(frac=1.)
    .pipe(
        (sns.relplot, "data"),
        x="Inter batch variance",
        y="Estimated variance",
        col="index",
        col_wrap=3,
        hue="step_mod_100",
        facet_kws={"sharey": False, "sharex": False},
    )
)

fg.set(xscale="log")
fg.set(yscale="log")
for ax in fg.axes.flatten():
    ax.axline((0, 0), (1, 1), color="red")

In [ ]:
(
    plot_data.assign(
        rel_diff=lambda x: (
            (x["Interbatch variance"] - x["Estimated variance"])
            / x["Interbatch variance"]
        ).abs()
    )
    .reset_index("index")
    .groupby(["step_mod_100", "index"])
    .mean()
    .pipe((sns.relplot, "data"), x="step_mod_100", y="rel_diff")
)
plt.yscale("log")


In [ ]:
plot_data = (
    pd.merge(
        pd.DataFrame(variance_estimated.numpy(), index=step)
        .reset_index()
        .melt(id_vars="index", value_name="variance_estimated"),
        pd.DataFrame(variance_inter_batch.numpy(), index=step)
        .reset_index()
        .melt(id_vars="index", value_name="variance_inter_batch"),
    )
    .assign(step_mod_100=lambda x: x["index"] % 100)
    .loc[lambda x: (x.variance_estimated > 1e-8) & (x.variance_inter_batch > 1e-8)]
)


In [ ]:
plot_data

In [ ]:
import numpy as np
import matplotlib.ticker as mticker
sns.set(style = "whitegrid")

fig = plt.figure(figsize=(10,8))
ax = plt.subplot(projection="3d")
x, y, z, c = plot_data[["index", "variance_inter_batch", "variance_estimated", "step_mod_100"]].to_numpy().T
def log_tick_formatter(val, pos=None):
    return f"10^{int(val):d}"
    
sc = ax.scatter(x, np.log10(y), np.log10(z), c=c)
ax.yaxis.set_major_formatter(mticker.FuncFormatter(log_tick_formatter))
ax.zaxis.set_major_formatter(mticker.FuncFormatter(log_tick_formatter))
ax.view_init(20, -30)
ax.set_xlabel("Step")
ax.set_ylabel("Estimated variance")
ax.set_zlabel("Interbatch variance")
plt.colorbar(sc)
plt.show()
